# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Hardware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

In [ ]:
versioninfo()

In [ ]:
using Pkg

In [ ]:
Pkg.add("Revise")
Pkg.add("DataFrames")
Pkg.add("BenchmarkTools")
Pkg.add(url="https://github.com/bwbioinfo/KEGGAPI.jl")
Pkg.add("TestImages")
Pkg.add("Images")
Pkg.add("FileIO")
Pkg.add("Colors")

In [ ]:
using Revise
using DataFrames
using BenchmarkTools
using KEGGAPI
using Images
using TestImages
using Colors

# Case 3: Identify compound in Kegg Database



### 1. Get compound information in Kegg Database

To determine if a compound is included in KEGG database and obtain its information, the function find takes as input the string "compound" and the name of the compound

In [ ]:
@time kegg_compound = KEGGAPI.find("compound", "chitin")
DataFrame(
  kegg_compound.data,
  kegg_compound.colnames
)

#### 1.1 Get compund information

In [ ]:
kegg_compound_info = KEGGAPI.kegg_get(["cpd:C00461"]);
split(kegg_compound_info[2][1], "\n")

### 2. Get image of the compund of interest

The get_image function is to download a any image, the imput is the compound number as "cpd:CXXXXX"

The save_image function is to save the figure in a png file. The input is a string with the name of the file and the extension ".png"

In [ ]:
@time kegg_image = KEGGAPI.get_image("cpd:C00461")
@time KEGGAPI.save_image(kegg_image, "chitin.png")

### 3. Visualize download compound

In [ ]:
img = load("chitin.png")

### 4. Identify thre reactions linked to the compound of interest

The ouput is one line for every time the compound is found

In [ ]:
@time kegg_reaction = KEGGAPI.link("reaction", "cpd:C00461")
DataFrame(
  kegg_reaction.data,
  kegg_reaction.colnames
)

#### 5. Get information about the reactions which chitin is involved

In [ ]:
@time kegg_reaction_info = KEGGAPI.kegg_get(kegg_reaction.data[2]);
split(kegg_reaction_info[2][1],"\n")

### 6. Get image of the reaction of interest

The get_image function is to download a any image, the imput is the reactio number as "rn:RXXXXX"

The save_image function is to save the figure in a png file. The input is a string with the name of the file and the extension ".png"

In [ ]:
@time kegg_image = KEGGAPI.get_image("rn:R01206")
@time KEGGAPI.save_image(kegg_image, "r01206.png")

### 7. Visualize download reaction

In [ ]:
img = load("r01206.png")